In [27]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [28]:
Customer.objects.filter(age=30).values('first_name')

<QuerySet [{'first_name': '영환'}, {'first_name': '보람'}, {'first_name': '은영'}, {'first_name': '상현'}, {'first_name': '정훈'}, {'first_name': '서연'}, {'first_name': '정식'}, {'first_name': '영수'}, {'first_name': '예진'}, {'first_name': '은지'}, {'first_name': '준영'}, {'first_name': '숙자'}, {'first_name': '현주'}, {'first_name': '성수'}, {'first_name': '상훈'}, {'first_name': '민재'}, {'first_name': '지아'}, {'first_name': '정순'}, {'first_name': '지원'}, {'first_name': '하은'}, '...(remaining elements truncated)...']>

In [14]:
# 나이가 30 이상인 사람의 인원수 (__gte, __gt, __lte, __lt)
# gte: ~이상
Customer.objects.filter(age__gte=30).count()

414

In [18]:
# 미성년자 인원수
Customer.objects.filter(age__lt=20).count()

213

In [21]:
Customer.objects.filter(age=30, last_name='김').values('last_name', 'age')

<QuerySet [{'last_name': '김', 'age': 30}, {'last_name': '김', 'age': 30}, {'last_name': '김', 'age': 30}, {'last_name': '김', 'age': 30}, {'last_name': '김', 'age': 30}, {'last_name': '김', 'age': 30}]>

In [23]:
# 나이가 30이면서 동시에 김씨인 사람의 수(and 연산)
Customer.objects.filter(age=30, last_name='김').count()

6

In [26]:
# 나이가 30이상이면서 동시에 김씨인 사람의 수(and 연산)
Customer.objects.filter(age__gte=30, last_name='김').count()

112

In [35]:
Customer.objects.filter(age__gte=30, last_name='강').count()

12

In [38]:
# 30 이상이거나, 강씨인 사람의 수
(Customer.objects.filter(age__gte=30) | Customer.objects.filter(last_name='강').values('age','last_name')).count()

425

In [41]:
# 이게 너무 귀찮으니까 Q의 등장

In [44]:
from django.db.models import Q
Customer.objects.filter(Q(age__gte=30) | Q(last_name='강')).count()

425

In [45]:
# % => 무엇이든
# _ => 반드시 그 자리에
# django orm에서는 % 만 사용 가능
# 그 이상의 패턴 비교를 하려면, 아예 정규 표현식(regex)를 사용한다.

In [47]:
# 02- 로 시작하는 전화번호 개수 (02-%)
Customer.objects.filter(phone__startswith='02-').count()

249

In [49]:
# 공식문서를 찾아보자

In [55]:
# 강원도민 중 황씨의 이름
Customer.objects.filter(country='강원도', last_name='황').values('first_name')

<QuerySet [{'first_name': '은정'}, {'first_name': '혜진'}, {'first_name': '경수'}, {'first_name': '준영'}]>

In [60]:
# 나이가 가장 많은 사람 10명(ORM이 자동으로 LIMIT, OFFSET 처리)
Customer.objects.order_by('-age')[:10]
Customer.objects.order_by('-age')[10:20]

<QuerySet [<Customer: Customer object (219)>, <Customer: Customer object (252)>, <Customer: Customer object (346)>, <Customer: Customer object (356)>, <Customer: Customer object (365)>, <Customer: Customer object (368)>, <Customer: Customer object (399)>, <Customer: Customer object (408)>, <Customer: Customer object (431)>, <Customer: Customer object (446)>]>

In [62]:
# 최빈 10명(디폴트는 오름차순)
Customer.objects.order_by('balance')[:10].values('balance')

<QuerySet [{'balance': 150}, {'balance': 150}, {'balance': 150}, {'balance': 150}, {'balance': 150}, {'balance': 160}, {'balance': 160}, {'balance': 160}, {'balance': 160}, {'balance': 170}]>

In [66]:
# 최빈 + 최연장자 10명: 돈없는데 늙으신분
# .values() 하면 전부 나옴
Customer.objects.order_by('balance', '-age')[:10].values('balance', 'age')

<QuerySet [{'balance': 150, 'age': 33}, {'balance': 150, 'age': 32}, {'balance': 150, 'age': 32}, {'balance': 150, 'age': 28}, {'balance': 150, 'age': 20}, {'balance': 160, 'age': 36}, {'balance': 160, 'age': 32}, {'balance': 160, 'age': 25}, {'balance': 160, 'age': 23}, {'balance': 170, 'age': 37}]>

In [71]:
# 가장 가난하고 늙으신 분 1명
Customer.objects.order_by('balance', '-age')[0]

<Customer: Customer object (852)>

In [74]:
# 성 / 이름 ㄱㄴㄷ 정렬 내림차순 기준 5번째
Customer.objects.order_by('-last_name','-first_name').values()[4]

{'id': 912,
 'first_name': '준영',
 'last_name': '황',
 'age': 16,
 'country': '강원도',
 'phone': '02-1030-5139',
 'balance': 6500}

## Annotate

In [80]:
from django.db.models import Count
from django.db.models.functions import Concat

In [81]:
Customer.objects.annotate(full_name=Concat('first_name', 'last_name')).values()[4]

{'id': 5,
 'first_name': '영환',
 'last_name': '차',
 'age': 30,
 'country': '충청북도',
 'phone': '011-2921-4284',
 'balance': 220,
 'full_name': '영환차'}

In [83]:
Customer.objects.annotate(Count('country')).values()[1]

{'id': 2,
 'first_name': '경희',
 'last_name': '이',
 'age': 36,
 'country': '경상남도',
 'phone': '011-9854-5133',
 'balance': 5900,
 'country__count': 1}